In [1]:
import pandas as pd

movies = pd.read_csv('./data/ml-1m/movies.dat', delimiter='::', engine='python', encoding='ISO-8859-1', names=['movieId', 'title', 'gneres'])
ratings = pd.read_csv('./data/ml-1m/ratings.dat', delimiter='::', engine='python', encoding='ISO-8859-1', names=['userId', 'movieId', 'rating', 'timestamp'])

movie_wiki = pd.read_csv('./data/crawled_data/movie_wiki.csv')
movie_imdb = pd.read_csv('./data/crawled_data/movie_imdb.csv')
movie_lbox = pd.read_csv('./data/crawled_data/movie_lbox.csv')

In [2]:
print(f'총 영화 개수는 {movies.shape[0]}개')
print(f'총 사용자 수는 {ratings["userId"].nunique()}명')
print(f'총 평점 개수는 {ratings.shape[0]}개')
print(f'평점이 부여된 영화 개수는 {ratings["movieId"].nunique()}개')

총 영화 개수는 3883개
총 사용자 수는 6040명
총 평점 개수는 1000209개
평점이 부여된 영화 개수는 3706개


In [3]:
# 평점은 5점 만점이며 1점 단위로 부여 가능
ratings['rating'].unique()

array([5, 3, 4, 2, 1], dtype=int64)

In [4]:
high_ratings = ratings['rating'].value_counts().get(4) + ratings['rating'].value_counts().get(5)

print(f'전체 평점 중에서 4점과 5점의 비중은 {round(high_ratings / ratings.shape[0], 4)}')

전체 평점 중에서 4점과 5점의 비중은 0.5752


In [5]:
# 4점 이상의 평점은 1, 4점 미만의 평점은 0으로 변환
ratings['rating'] = ratings['rating'].apply(lambda x: 1 if x >= 4 else 0)

# userID -> timestamp 순으로 정렬
ratings = ratings.sort_values(by=['userId', 'timestamp']).reset_index(drop=True)

In [6]:
# 위키피디아; 위키피디아만 plots에 결측치가 존재함
plots_wiki = movie_wiki[movie_wiki['plots'] != 'no plots']
plots_wiki = plots_wiki.dropna(subset=['plots'])

no_plots_wiki = movie_wiki[(movie_wiki['plots'].isna()) | (movie_wiki['plots'] == 'no plots')]

print(f'위키피디아에서 줄거리가 수집된 영화 개수는 {plots_wiki.shape[0]}개')
print(f'위키피디아에서 줄거리가 수집되지 않은 영화 개수는 {no_plots_wiki.shape[0]}개')

위키피디아에서 줄거리가 수집된 영화 개수는 3421개
위키피디아에서 줄거리가 수집되지 않은 영화 개수는 462개


In [7]:
# 위키피디아에서 줄거리가 수집되지 않은 영화는 imdb에서 크롤링한 것으로 대체
temp = pd.merge(left=no_plots_wiki.loc[:, ['movieId', 'title', 'genres']], 
                right=movie_imdb.loc[:, ['movieId', 'plots']], 
                how='inner', 
                on='movieId')

plots_temp = temp[(temp['plots'] != 'no result') & (temp['plots'] != 'no synopsis')]
no_plots_temp = temp[(temp['plots'] == 'no result') | (temp['plots'] == 'no synopsis')]

print(f'imdb에서 줄거리가 수집된 영화의 개수는 {plots_temp.shape[0]}개')
print(f'위키피디아와 imdb에서 줄거리가 수집되지 못한 영화의 개수는 {no_plots_temp.shape[0]}개')

imdb에서 줄거리가 수집된 영화의 개수는 71개
위키피디아와 imdb에서 줄거리가 수집되지 못한 영화의 개수는 391개


In [8]:
# 위키피디아와 imdb에서 줄거리가 수집되지 않은 영화는 lbox에서 크롤링한 것으로 대체
temp2 = pd.merge(left=no_plots_temp.loc[:, ['movieId', 'title', 'genres']], 
                 right=movie_lbox.loc[:, ['movieId', 'plots']], 
                 how='inner', 
                 on='movieId')

plots_temp2 = temp2[(temp2['plots'] != 'no results') & (temp2['plots'] != 'no synopsis')]
no_plots_temp2 = temp2[(temp2['plots'] == 'no results') | (temp2['plots'] == 'no synopsis')]

print(f'lbox에서 줄거리가 수집된 영화의 개수는 {plots_temp2.shape[0]}개')
print(f'모든 플랫폼에서 줄거리가 수집되지 못한 영화의 개수는 {no_plots_temp2.shape[0]}개')

lbox에서 줄거리가 수집된 영화의 개수는 285개
모든 플랫폼에서 줄거리가 수집되지 못한 영화의 개수는 106개


In [9]:
final_df = pd.concat(objs=[plots_wiki, plots_temp, plots_temp2], axis=0)
final_df = final_df.reset_index(drop=True)

final_df.to_csv('./data/crawled_data/plots_crawled.csv', index=False)

In [10]:
missing_id = no_plots_temp2['movieId']

ratings = ratings[~ratings['movieId'].isin(missing_id)]
ratings = ratings.reset_index(drop=True)
ratings.to_csv('./data/ml-1m/ratings.csv', index=False)

movies = movies[~movies['movieId'].isin(missing_id)]
movies = movies.reset_index(drop=True)

print(f'최종적으로 분석에 사용될 평점 히스토리의 개수: {ratings.shape[0]}개')
print(f'최종적으로 분석에 사용될 영화의 개수: {movies.shape[0]}개')

최종적으로 분석에 사용될 평점 히스토리의 개수: 997772개
최종적으로 분석에 사용될 영화의 개수: 3777개
